### This file is run only once to download the data from its source location and store in S3

In [4]:
import pandas as pd
import re
import boto3


In [5]:
!mkdir ../data
!wget -O ../data/WISDM_ar_latest.tar.gz http://www.cis.fordham.edu/wisdm/includes/datasets/latest/WISDM_ar_latest.tar.gz
!tar -zxf ../data/WISDM_ar_latest.tar.gz -C ../data 

mkdir: cannot create directory ‘../data’: File exists
--2020-03-24 15:55:21--  http://www.cis.fordham.edu/wisdm/includes/datasets/latest/WISDM_ar_latest.tar.gz
Resolving www.cis.fordham.edu (www.cis.fordham.edu)... 150.108.68.29
Connecting to www.cis.fordham.edu (www.cis.fordham.edu)|150.108.68.29|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11404612 (11M) [application/x-gzip]
Saving to: ‘../data/WISDM_ar_latest.tar.gz’

../data/WISDM_ar_la 100%[===================>]  10.88M  15.5MB/s    in 0.7s    

2020-03-24 15:55:22 (15.5 MB/s) - ‘../data/WISDM_ar_latest.tar.gz’ saved [11404612/11404612]



In [10]:
!aws s3api create-bucket --bucket sagemaker-wisdm2 --region us-east-1

{
    "Location": "/sagemaker-wisdm2"
}


In [11]:
bucket = "sagemaker-wisdm2"
data_location = 's3://{}'.format(bucket)

s3 = boto3.resource('s3')

In [8]:
columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv('../data/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt', header= None, names = columns)

df['z-axis'] = [re.sub(';','', str(df.loc[row,'z-axis'])) for row in range(len(df))]    

### Data is split into 12 files to simulate different data drops

In [12]:
length = int(len(df)/12)

for i in range(12):
    init=int(i*length)
    fin = int(init+length)
    prefix = '../'
    fname = 'data/df' + str(i) + '.csv'
    df[init:fin].to_csv(str(prefix+fname), index=False, header=True)
    s3.Bucket(bucket).upload_file(str(prefix+fname), fname)
    print(fname)

data/df0.csv
data/df1.csv
data/df2.csv
data/df3.csv
data/df4.csv
data/df5.csv
data/df6.csv
data/df7.csv
data/df8.csv
data/df9.csv
data/df10.csv
data/df11.csv
